# Computación cuántica desde tu casa
Talleres que molan. Facultad de Filosofía, Universidad Complutense de Madrid. 27/ABR/2020.

(c) Alejandro Pozas Kerstjens

www.alexpozas.com

physics [a] alexpozas.com

---
## Introducción
La física cuántica es el conjunto de principios físicos que rigen el comportamiento de los átomos y de las partículas subatómicas. Desarrollada durante el primer tercio del siglo XX por científicos como Planck, Einstein, Bohr o Heisenberg, es una teoría que recibió gran resistencia tanto por la comunidad científica en general como incluso por sus propios creadores. La razón de esto es que, para poder describir con precisión algunos fenómenos físicos que se observan en el laboratorio, la física cuántica requiere asumir unas premisas muy diferentes a lo que nos dictaría la intuición. Sin embargo, con el paso del tiempo, se ha observado que la física cuántica es **la teoría más precisa jamás desarrollada**, pudiendo predecir, por ejemplo, cantidades como el momento dipolar del electrón con una incertidumbre solamente de una parte en 10 billones (españoles). Esto es equivalente a determinar si estás en un extremo u otro del Parque del Oeste en Madrid desde la estrella vecina Proxima Centauri.

Tal y como hemos dicho, esta exactitud a la hora de realizar predicciones no es gratis. La física cuántica requiere de una naturaleza indeterminista, donde los sistemas no tienen propiedades definidas hasta que se les realiza una medición, el vacío y la nada son conceptos totalmente diferentes, o existen cantidades que no pueden medirse simultáneamente con una precisión arbitraria. Estas características son consecuencia de que la física cuántica es una teoría cuyos postulados fundamentales son en gran medida más matemáticos que físicos (en comparación con, por ejemplo, la teoría de la relatividad, cuyo postulado fundamental _La velocidad de la luz es la misma observada desde cualquier sistema de referencia inercial_ tiene una gran conexión con el mundo físico). De hecho, la interpretación de cuáles son las verdaderas manifestaciones de los postulados cuánticos en el mundo real son un objeto de gran debate y, de hecho, existe un gran número de escuelas que tienen diferentes _interpretaciones_ acerca del significado de los postulados, entre las que se encuentran la [escuela de Copenhague](https://en.wikipedia.org/wiki/Copenhagen_interpretation) (mayoritaria a día de hoy), la interpretación de la [_onda piloto_](https://en.wikipedia.org/wiki/De_Broglie%E2%80%93Bohm_theory) de Bohm y de Broglie, la interpretación de los [_muchos mundos_](https://en.wikipedia.org/wiki/Many-worlds_interpretation) de Everett, o el [_bayesianismo cuántico_](https://en.wikipedia.org/wiki/Quantum_Bayesianism) (o QBismo) de Fuchs, entre muchos otros.



### En este taller
Este taller tiene dos objetivos fundamentales: por un lado, hacer una introducción a los principios fundamentales de la física cuántica, y por otro, mostrar que sus conclusiones se pueden poner a prueba desde casa.

Para ello, tomaremos un punto de vista muy particular: desde el primer momento utilizaremos programas informáticos para ir jugando en tiempo real con los conceptos y las explicaciones. En concreto, utilizaremos la librería de programación [Qiskit](http://www.qiskit.org), de código abierto y desarrollada por IBM para tener una interfaz con sus ordenadores cuánticos.

Una vez terminada la explicación de los conceptos fundamentales, exploraremos de manera práctica las consecuencias de los postulados cuánticos mediante el *teorema de Bell*, que demuestra que el comportamiento de partículas cuánticas no puede ser explicado por ninguna teoría que sea a la vez local y realista. Finalmente, veremos unos pequeños apuntes acerca de cómo controlar experimentos cuánticos reales desde la comodidad del sofá.

### Comencemos
El primer paso necesario es instalar todas las librerías necesarias para poder ejecutar los programas que construiremos. Además, crearemos un par de funciones que iremos utilizando a lo largo del taller.

In [ ]:
!pip install qiskit=0.16.2

In [ ]:
import matplotlib.pyplot             # Para gráficos
%matplotlib inline
from qiskit import Aer, IBMQ         # Simulador y acceso a los chips cuánticos
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister    # Para crear circuitos
from qiskit import execute           # Para ejecutar circuitos
from qiskit.visualization import plot_bloch_multivector    # Para visualización
from qiskit.extensions import Initialize    # Rutina de inicialización
%config InlineBackend.figure_format = 'svg'

from numpy import pi as π
from numpy import append, array, sqrt
from numpy.random import random, randint

In [ ]:
def estado_aleatorio(nqubits):
    """Crea un estado aleatorio de n qubits"""
    partes_reales      = array([])
    partes_imaginarias = array([])
    for amplitud in range(2**nqubits):
        partes_reales      = append(partes_reales, 2*random()-1)
        partes_imaginarias = append(partes_imaginarias, 2*random()-1)
    amps = partes_reales + 1j*partes_imaginarias
    # Normalización
    norma_cuadrada = 0
    for a in amps:
        norma_cuadrada += abs(a)**2
    amps /= sqrt(norma_cuadrada)
    return amps

def formatea_estado(vector_estado):
    """Dado un vector de estado de un qubit, lo formatea en la manera estándar"""
    estados_base = ["|0>", "|1>"]
    estado = ""
    for amplitud, estado_base in zip(vector_estado, estados_base):
        if abs(amplitud - 1) < 1e-8:
            estado += "+" + estado_base
        elif abs(amplitud) < 1e-8:
            continue
        else:
            if str(amplitud)[0] == "-":
                estado += str(round(amplitud, 4)) + estado_base
            else:
                estado += "+" + str(round(amplitud, 4)) + estado_base
    return estado[1:] if estado[0] != "-" else estado

## Los ladrillos del laboratorio
A nivel fundamental, un experimento físico tiene tres componentes esenciales. Por un lado, son necesarios unos **sistemas físicos** sobre los que se realizará el experimento. A estos sistemas se les realiza una serie de **transformaciones**, tras las cuales se procede a hacer **medidas** de sus propiedades. El siguiente paso es, pues, ir definiendo estos objetos en nuestros programas.

### Circuitos y bits cuánticos
La librería `qiskit` que vamos a utilizar está enfocada, sobre todo, a aplicaciones de computación, de modo que los sistemas físicos que trataremos serán **qubits**, o bits cuánticos (más adelante veremos en detalle qué son los qubits y en qué se diferencian de los bits clásicos), los cuales son manipulados y medidos en circuitos cuánticos.

La manera de crear un circuito con un número determinado de qubits es a través de la función `QuantumCircuit`. De momento, vamos a contentarnos con un solo qubit:

In [ ]:
n_qubits = 1
qubit    = QuantumCircuit(n_qubits)

Pictóricamente, cada qubit representará una línea sólida en el circuito, a las cuales se irán aplicando transformaciones, que normalmente aparecen en forma de cajas. Para ver la representación de un circuito particular, podemos llamar a su atributo `draw`.

In [ ]:
qubit.draw(output="mpl")

### Operaciones/transformaciones
A continuación veremos cómo aplicar transformaciones a los qubits, que conforman la parte central del experimento. El conjunto de transformaciones básicas que se puede realizar a uno o varios qubits se pueden ver en la [documentación](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.html), aunque una descripción más amable de las transformaciones más comunes se puede encontrar [aquí](https://qiskit.org/textbook/ch-states/single-qubit-gates.html). Sin embargo, es importante mencionar que este conjunto de transformaciones básicas es un [conjunto universal](https://en.wikipedia.org/wiki/Quantum_logic_gate#Universal_quantum_gates), es decir, que las transformaciones básicas se pueden concatenar para dar lugar a cualquier transformación que uno desee.

En Qiskit, las transformaciones se aplican mediante llamadas a funciones atributo del ``QuantumCircuit``. Empecemos por algo sencillo. Tomemos nuestro qubit anterior, y apliquemos una *puerta $X$*:

In [ ]:
qubit.x(qubit=0)    # Aplicamos una puerta X en el qubit 0
qubit.draw("mpl")

### Medidas y registros
Tras hacer las transformaciones, queremos medir las propiedades de los qubits. Estas medidas se llevan a cabo llamando a la función `measure`, que toma dos argumentos: por un lado, el qubit que queremos medir, y por otro, un registro clásico donde guardar el resultado de la medida.

Veamos un ejemplo: tomaremos un qubit y haremos primero una medida, cuyo resultado almacenaremos en un registro clásico, después le aplicaremos una transformación, y finalmente volveremos a medirlo, almacenando el resultado en otro registro diferente.

In [ ]:
circuito_x = QuantumCircuit(QuantumRegister(1), ClassicalRegister(1), ClassicalRegister(1))
circuito_x.measure(qubit=0, cbit=0)    # Medimos el estado del qubit
circuito_x.x(qubit=0)                  # Aplicamos la puerta X en el qubit 0
circuito_x.measure(0, 1)               # Medimos el estado del qubit de nuevo
circuito_x.draw(output="mpl")

### Ejecutando los experimentos
El último paso que nos queda es, una vez que hemos diseñado nuestro experimento, llevarlo a cabo.
Esto se hace mediante la función `execute` que, a su vez, necesita de la especificación de una plataforma en la cual se llevará a cabo el experimento. Existen tres plataformas de gran utilidad:

- El ```statevector_simulator```. Es un simulador de estados cuánticos que utiliza álgebra lineal para computar el resultado final, aplicando una representación matemática correspondiente al circuito y dando lugar al estado final de los qubits.

- El ```qasm_simulator```. Es un simulador clásico de un procesador cuántico real. Produce una simulación de un experimento completo, de modo que el resultado no es el estado final de los qubits, sino los registros clásicos donde se han ido guardando los resultados de las medidas que realizamos a lo largo del experimento.

- Chips cuánticos reales. Cada uno tiene su propio nombre, y veremos más adelante cómo llamar a cada uno de ellos.

Los simuladores se encuentran en el módulo ```Aer``` de ```qiskit```, mientras que para acceder a los chips reales necesitamos el módulo ```IBMQ```. Comencemos con los simuladores.

In [ ]:
simulador_de_vectores = Aer.get_backend("statevector_simulator")
simulador_de_chip     = Aer.get_backend("qasm_simulator")

Ahora podemos ver, por ejemplo, cuál es el resultado de ejecutar el circuito anterior, `circuito_x`.

In [ ]:
execute(circuito_x, simulador_de_chip).result().get_counts()

Este resultado indica que `circuito_x` ha sido ejecutado un total de 1024 veces, y que en todas ellas la información grabada en el primer registro ha sido un 0 y en el segundo registro un 1 (sí, en `qiskit` los registros se leen de derecha a izquierda)

Una vez que tenemos los ladrillos, podemos comenzar a ver las propiedades que hacen de la física cuántica una teoría tan singular.
## Propiedades cuánticas fundamentales
---
### El estado vector de un sistema cuántico
Acabamos de ver que el `qasm_simulator` simula un experimento real de manera exacta: toma una serie de qubits, los transforma, los mide, y lo que muestra al final es el registro de resultados de dichas medidas. Si esto es lo más cercano que tenemos a un experimento real, ¿por qué hemos invocado también al `statevector_simulator`? ¿Cuál es el resultado al ejecutar el experimento en dicho simulador? Veámoslo.

In [ ]:
qubit_statevector = execute(qubit, simulador_de_vectores)
print(qubit_statevector.result().get_statevector())

El resultado es un vector (una colección de números) que, no casualmente, se llama *vector de estado*, que determina el estado en el que se encuentra el sistema. Cada qubit tiene dos estados posibles, que normalmente se denotan por $|0\rangle$ y $|1\rangle$. Cada una de las componentes del vector de estado se llama *amplitud de probabilidad*, y veremos un poco más adelante qué significa esto. En general, el estado de un qubit puede describirse mediante $|\psi\rangle=(c_0, c_1)=c_0|0\rangle + c_1|1\rangle$, donde las amplitudes de probabilidad $c_0$ y $c_1$ son números complejos que satisfacen $|c_0|^2+|c_1|^2=1$. En el caso de arriba tenemos que $c_0=0$ y $c_1=1$, de modo que el qubit está en el estado $|\psi\rangle = |1\rangle$.

El vector de estado es una caracterización unívoca del estado de nuestro sistema físico, y determina, entre otros, los resultados que se obtendrán al realizar medidas en él. Como ejemplo de esto, veamos qué ocurre al medir los estados $|0\rangle$ y $|1\rangle$:

In [ ]:
estado_0 = QuantumCircuit(1, 1)
# Los qubits, al iniciar un circuito, están todos en el estado |0>
# Consecuentemente, no tenemos que hacer nada más que medirlo
estado_0.measure(0, 0)
# Y ejecutamos el programa
execute(estado_0, simulador_de_chip).result().get_counts()

In [ ]:
estado_1 = QuantumCircuit(1, 1)
# Preparamos el qubit en el estado |1> aplicando una puerta X
estado_1.x(0)
# Y ahora medimos
estado_1.measure(0, 0)
# Finalmente, ejecutamos el experimento
execute(estado_1, simulador_de_chip).result().get_counts()

Siempre que medimos un sistema que está en su estado $|0\rangle$, el resultado de la medida es 0, mientras que siempre que medimos un sistema que está en su estado $|1\rangle$, el resultado de la medida es 1. Estos estados son también conocidos como *estados clásicos*, y son los únicos estados en los que se puede preparar un bit clásico.

Al contrario, existen muchos sistemas cuánticos (o mejor dicho, muchos estados posibles de un mismo sistema) que dan lugar a los mismos resultados al medirlos. Consecuentemente, el vector de estado es una descripción más fundamental del estado de un sistema que el resultado de medidas. Sin embargo, dado que no involucra medidas, **el vector de estado no es una cantidad a la que se pueda acceder directamente en un experimento**. Esto abre una gran pregunta que tiene diferente respuesta en las distintas escuelas, la pregunta de [*qué es lo que realmente representa el vector de estado*](https://arxiv.org/abs/1111.3328): ¿Es un objeto real? ¿Es una representación de nuestro conocimiento del sistema?

Existe una manera muy visual de estudiar estados cuánticos, mediante la denominada [Esfera de Bloch](https://en.wikipedia.org/wiki/Bloch_sphere). Ésta es una esfera en la que los vectores de estado están representados por flechas que se originan en el centro y apuntan hacia afuera.

Veamos una serie de ejemplos de cómo distintos vectores de estado se ven en la esfera de Bloch. Haremos tres ejemplos: el estado $|0\rangle$, el estado $|1\rangle$, y un estado aleatorio.

In [ ]:
diferentes_estados = QuantumCircuit(3)
# Al iniciar un circuito, los qubits están en |0>, de modo que al primero no le hacemos nada

# Aplicamos una puerta X al segundo qubit para pasarlo del estado |0> al |1>
diferentes_estados.x(1)

# Preparamos un estado aleatorio en el tercer qubit
aleatorio = Initialize(estado_aleatorio(1))
diferentes_estados.append(aleatorio, [2])

# Y dibujamos los tres en la esfera de Bloch
plot_bloch_multivector(execute(diferentes_estados, simulador_de_vectores).result().get_statevector())

Los dos primeros estados son los únicos en los que se puede encontrar un bit *clásico* (una moneda solamente puede estar con la cara hacia arriba o con la cara hacia abajo, un interruptor solamente puede estar encendido o apagado...), pero como se ve en la tercera esfera, los bits (y todos los sistemas) cuánticos se pueden encontrar en *estados de superposición*. Esta es una gran diferencia entre la física clásica y la cuántica, que está detrás de muchas de las aplicaciones de la física cuántica. Veamos un poquito más acerca de estos estados de superposición, con unos ejemplos muy particulares.

### Superposición
Vamos a crear estados en superposición utilizando la llamada *puerta de Hadamard*, una transformación que transforma el estado $|0\rangle$ en el estado $\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$, y el estado $|1\rangle$ en el estado $\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)$. Veámoslo explícitamente:

In [ ]:
# Aplicación de la puerta de Hadamard al estado |0>
hadamard_desde_cero = QuantumCircuit(1)
hadamard_desde_cero.h(0)
resultado = execute(hadamard_desde_cero, simulador_de_vectores).result().get_statevector()
_, x = plt.subplots()
hadamard_desde_cero.draw("mpl", ax=ax)
print(resultado)
plot_bloch_multivector(resultado)

In [ ]:
# Aplicación de la puerta de Hadamard al estado |1>
hadamard_desde_uno = QuantumCircuit(1)
hadamard_desde_uno.x(0)
hadamard_desde_uno.h(0)
resultado = execute(hadamard_desde_uno, simulador_de_vectores).result().get_statevector()
_, x = plt.subplots()
hadamard_desde_uno.draw("mpl", ax=ax)
print(resultado)
plot_bloch_multivector(resultado)

¿Qué pasará cuando hagamos una medición en estos estados? Veámoslo

In [ ]:
hadamard_desde_cero.add_register(ClassicalRegister(1))
hadamard_desde_cero.measure(0, 0)
print(execute(hadamard_desde_cero, simulador_de_chip, shots=10000).result().get_counts())

hadamard_desde_uno.add_register(ClassicalRegister(1))
hadamard_desde_uno.measure(0, 0)
print(execute(hadamard_desde_uno, simulador_de_chip, shots=10000).result().get_counts())

Parece ser que, en ambos casos, más o menos la mitad de las veces el resultado de la medida es $0$, y la otra mitad de las veces es $1$. Esto es debido a que **la física cuántica describe una naturaleza probabilista**. Cuando un qubit (o cualquier sistema cuántico) se encuentra en un estado de superposición y se realiza una medición sobre él, el resultado de dicha medición es una de las posibles opciones de la superposición, con una probabilidad asociada a la correspondiente amplitud de probabilidad. Esto es, si realizamos una medida en un qubit preparado en el estado $|\psi\rangle=c_0|0\rangle + c_1|1\rangle$, el resultado de dicha medida será $0$ con una probabilidad $|c_0|^2$, y $1$ con una probabilidad $|c_1|^2$. En el caso de las superposiciones tras la puerta de Hadamard, estas probabilidades son $\left|\pm\frac{1}{\sqrt{2}}\right|^2=\frac{1}{2}$.

### Colapso del vector de estado
¿Y qué pasa con el vector de estado tras la medida? ¿Sufre algún cambio, o sigue igual? Veámoslo explícitamente, ejecutando el programa `hadamard_desde_cero` que hemos creado anteriormente.

In [ ]:
hadamard_desde_cero.draw("mpl")

In [ ]:
for _ in range(10):
    experimento = execute(hadamard_desde_cero, simulador_de_vectores, memory=True)
    resultado   = experimento.result()
    resultado_de_la_medida = resultado.get_memory()
    estado_vector_final    = resultado.get_statevector()
    print("El resultado de la medida fue " + resultado_de_la_medida[0]
        + " y el estado tras la medida es " + formatea_estado(estado_vector_final))

Cuando realizamos una medida, **el estado cambia al correspondiente al resultado de la medida**. Este fenómeno se llama [colapso de la función de onda](https://es.wikipedia.org/wiki/Colapso_de_la_funci%C3%B3n_de_onda), y es un proceso puramente cuántico sin análogo en el mundo clásico. En ámbitos como el de la computación cuántica, lo que importa es saber que *realizar medidas destruye las superposiciones*, de tal manera que los efectos cuánticos de un sistema se pierden al medirlo. Esto da lugar a fenómenos como el [**efecto Zenón cuántico**](https://en.wikipedia.org/wiki/Quantum_Zeno_effect), el cual consiste en "congelar" la evolución de un sistema a base de medirlo continuamente. Por otro lado, el colapso de la función de onda tiene una interpretación muy sencilla en las escuelas epistémicas: si el vector de estado solamente nos proporciona información acerca del mundo, al realizar observaciones y actualizar nuestro conocimiento del mundo el vector de estado también debe actualizarse.

### Entrelazamiento
Si nos movemos del análisis de un sistema cuántico al análisis de varios, el primer fenómeno que aparece es el [entrelazamiento cuántico](https://es.wikipedia.org/wiki/Entrelazamiento_cu%C3%A1ntico). El estado de un sistema compuesto por dos o más sistemas cuánticos se denomina entrelazado si no puede describirse mediante estados aislados de los componentes individuales. Un ejemplo de esto es el estado $\frac{1}{\sqrt{2}}(|00\rangle+|11\rangle)$.

¿Qué ocurre cuando realizamos medidas en uno de estos estados entrelazados?

In [ ]:
def crear_par_entrelazado(circuito, a, b):
    """Crea un estado de Bell entre los qubits a y b de"""
    circuito.h(a)    # Pone el qubit a en estado de superposición |+> = |0> + |1>
    circuito.cx(a, b) # CNOT con a siendo control y b siendo objetivo
    return circuito

par_entrelazado = crear_par_entrelazado(QuantumCircuit(2, 2), 0, 1)
par_entrelazado.measure(0, 0)
par_entrelazado.measure(1, 1)

execute(par_entrelazado, simulador_de_chip, shots=10000).result().get_counts()

En efecto, los resultados de las medidas en los distintos sistemas están correlacionadas. Sin embargo, estas correlaciones son *más fuertes que cualquier correlación que se puede generar entre sistemas clásicos*. Qué significa esta afirmación la veremos a continuación, al estudiar el teorema de Bell.

Ya hemos visto una serie de conceptos y propiedades fundamentales de la física cuántica: el vector de estado, la superposición, el colapso del vector de estado, la naturaleza probabilista, y el entrelazamiento. Ahora, vamos a ver cómo estas propiedades permiten revelar un aspecto muy sorprendente de la naturaleza.

Ejemplo: El teorema de Bell
--
El [teorema de Bell](https://en.wikipedia.org/wiki/Bell%27s_theorem) ha llegado a ser determinado como _el resultado más profundo en ciencia_. En pocas palabras, el teorema de Bell establece una propiedad que debe ser satisfecha por cualquier sistema físico que cumpla dos características muy deseables en la naturaleza: (i) poseer propiedades que existen independiente de si se miden o no (*realismo*), y (ii) que la realidad en un punto no se vea afectada por medidas simultáneas en otros lugares alejados (*localidad*).

Sin embargo, existen sistemas cuánticos y medidas realizadas en dichos sistemas que no cumplen el teorema desarrollado por Bell. Esto significa, y ésta es la gran consecuencia del teorema de Bell, que **la naturaleza, a nivel microscópico, es no-realista, no-local, o ambas**.

En primer lugar veremos la formulación del teorema, y más tarde diseñaremos un experimento cuántico en el que se observará que dicho teorema no se cumple.

### Formulación del teorema
Imaginemos dos amigos, Alice y Bob, que deciden jugar un juego de respuestas correlacionadas. El juego tiene las siguientes reglas:

- El juego tiene varias rondas, en cada una de las cuales cada jugador responderá a una de un posible de dos preguntas, por ejemplo: "Dime un color" y "Dime una forma".
- Cada pregunta tiene solo dos respuestas posibles. Para "Dime un color" las respuestas son "Verde" y "Morado", y para "Dime una forma", "Triángulo" y "Círculo".
- Al comenzar el juego, cada participante se va a una sala aislada diferente, y una vez el juego ha empezado, no pueden comunicarse con el otro en ningún momento.
- Los dos jugadores pueden sin embargo, antes de comenzar a jugar, juntarse para acordar una estrategia previa de respuestas.
- En cada ronda, cada jugador lanza una moneda al aire, que determinará la pregunta que debe responder. Después, puede utilizar la estrategia acordada para responder a dicha pregunta.
- El objetivo es el siguiente: cuando ambas preguntas sean "Dime un color", las respuestas dadas por Alice y Bob deben ser las mismas el máximo número de veces. Por el contrario, cuando ambas preguntas sean "Dime una forma", las respuestas deben ser diferentes el mayor número de veces. Cuando las preguntas de Alice y Bob sean diferentes, se debe hacer que las respuestas Verde-Triángulo y Morado-Círculo sean más frecuentes que el resto de combinaciones.

Este juego cumple los dos requerimientos de realismo y localidad. Por un lado, la localidad está asegurada por el hecho de que los jugadores no pueden comunicarse una vez comienza el juego, de modo que Alice no puede cambiar su respuesta en función de la pregunta que haya tenido que responder Bob o lo que haya respondido, y viceversa. Por otro lado, la realidad está reflejada en la estrategia previa que siguen (también llamada *variable oculta*): la estrategia para responder a las preguntas está determinada antes de que las preguntas sean formuladas.

Para ver cuál es la mejor estrategia que Alice y Bob pueden seguir, vamos a formular el objetivo en notación matemática. El objetivo del juego es maximizar el siguiente objeto:

$$ \mathcal{B}=\langle A_C B_C\rangle + \langle A_F B_C\rangle + \langle A_C B_F\rangle - \langle A_F B_F\rangle $$

donde

$$ \langle A_C B_C\rangle = \frac{N^{CC}_{VV} + N^{CC}_{MM} - N^{CC}_{VM} - N^{CC}_{MV}}{N^{CC}_{VV} + N^{CC}_{MM} + N^{CC}_{VM} + N^{CC}_{MV}}$$

$$ \langle A_F B_C\rangle = \frac{N^{FC}_{\triangle V} + N^{FC}_{\bigcirc M} - N^{FC}_{\triangle M} - N^{FC}_{\bigcirc V}}{N^{FC}_{\triangle V} + N^{FC}_{\bigcirc M} + N^{FC}_{\triangle M} + N^{FC}_{\bigcirc V}}$$

$$ \langle A_C B_F\rangle = \frac{N^{CF}_{V\triangle} + N^{CF}_{M\bigcirc} - N^{CF}_{M\triangle} - N^{CF}_{V\bigcirc}}{N^{CF}_{V\triangle} + N^{CF}_{M\bigcirc} + N^{CF}_{M\triangle} + N^{CF}_{V\bigcirc}}$$

$$ \langle A_F B_F\rangle = \frac{N^{FF}_{\triangle\triangle} + N^{FF}_{\bigcirc\bigcirc} - N^{FF}_{\triangle\bigcirc} - N^{FF}_{\bigcirc\triangle}}{N^{FF}_{\triangle\triangle} + N^{FF}_{\bigcirc\bigcirc} + N^{FF}_{\triangle\bigcirc} + N^{FF}_{\bigcirc \triangle}}$$

Pongamos un ejemplo de estrategia que pueden seguir Alice y Bob. Digamos que, cada vez que alguno obtenga la pregunta "Dime un color", siempre responderá "Verde", y cada vez que obtengan la pregunta "Dime una forma", siempre se responderá "Triángulo". Esto hace que $\langle A_C B_C\rangle=1$, $\langle A_F B_C\rangle=1$, $\langle A_C B_F\rangle=1$ y $\langle A_F B_F\rangle=1$, dando lugar a un valor total de $\mathcal{B}=2$.

El teorema de Bell establece que, independientemente de lo compleja que sea la estrategia determinada por Alice y Bob antes de comenzar el juego, jamás podrán superar este valor de 2 (y, análogamente, jamás podrán obtener un valor inferior a -2). En otras palabras, el teorema de Bell determina que estrategias locales y realistas satisfacen $-2\leq\mathcal{B}\leq 2$.

### Una violación cuántica del teorema
¿Qué ocurre si, en lugar de acordar una estrategia previa de respuestas, Alice y Bob decidieran juntarse para compartir una colección de sistemas cuánticos en estados entrelazados $|\psi\rangle=\frac{1}{\sqrt{2}}(|00\rangle+|11\rangle)$? Entonces, pueden acordar llevar a cabo la siguiente estrategia:

- Cuando Alice reciba la pregunta "Dime un color", ella primero aplicará una puerta $X$ en su qubit y luego lo medirá, respondiendo "Verde" si el resultado obtenido es 0, y "Morado" si el resultado es 1.
- Cuando Alice reciba la pregunta "Dime una forma", primero aplicará una puerta $X$ en su qubit, después una puerta de Hadamard, y luego lo medirá, respondiendo "Triángulo" si el resultado obtenido es 0, y "Círculo" si el resultado obtenido es 1.
- Cuando Bob reciba la pregunta "Dime un color", él aplicará una rotación a su qubit de 135° alrededor del eje Y, después lo medirá, y responderá "Verde" si el resultado obtenido es 0, y "Morado" si el resultado es 1.
- Cuando Bob reciba la pregunta "Dime una forma", aplicará una rotación a su qubit de 225° alrededor del eje Y, después lo medirá, y responderá "Triángulo" si el resultado obtenido es 0, y "Círculo" si el resultado es 1.

Veamos cuál es el valor de $\mathcal{B}$ para esta estrategia:

In [ ]:
qr = QuantumRegister(2)
a = ClassicalRegister(1)
b = ClassicalRegister(1)

plataforma = Aer.get_backend('qasm_simulator')

In [ ]:
num_rondas = 1000

A0B0   = []
A0B1   = []
A1B0   = []
A1B1   = []
for _ in range(num_rondas):
    # Creamos un par entrelazado entre Alice y Bob
    experimento = QuantumCircuit(qr, a, b)
    crear_par_entrelazado(experimento, 0, 1)
    # Escogemos aleatoriamente las medidas que hará cada uno
    medida_A = randint(2)
    medida_B = randint(2)
    # Operación de Alice
    if medida_A == 1:
        experimento.h(0)
    experimento.x(0)
    experimento.measure(0, 0)
    # Operación de Bob
    experimento.ry(3*π / 4 + medida_B*π/2, 1)
    experimento.measure(1, 1)
    
    resultado = execute(experimento, plataforma, shots=1).result().get_counts()
    if medida_A == 0 and medida_B == 0:
        A0B0 += list(resultado)
    elif medida_A == 0 and medida_B == 1:
        A0B1 += list(resultado)
    elif medida_A == 1 and medida_B == 0:
        A1B0 += list(resultado)
    elif medida_A == 1 and medida_B == 1:
        A1B1 += list(resultado)

In [ ]:
A0B0 = array(A0B0)
A0B1 = array(A0B1)
A1B0 = array(A1B0)
A1B1 = array(A1B1)

A0B0_media = (sum(A0B0 == '0 0') - sum(A0B0 == '0 1') - sum(A0B0 == '1 0') + sum(A0B0 == '1 1')) / len(A0B0)
A0B1_media = (sum(A0B1 == '0 0') - sum(A0B1 == '0 1') - sum(A0B1 == '1 0') + sum(A0B1 == '1 1')) / len(A0B1)
A1B0_media = (sum(A1B0 == '0 0') - sum(A1B0 == '0 1') - sum(A1B0 == '1 0') + sum(A1B0 == '1 1')) / len(A1B0)
A1B1_media = (sum(A1B1 == '0 0') - sum(A1B1 == '0 1') - sum(A1B1 == '1 0') + sum(A1B1 == '1 1')) / len(A1B1)

print(A0B0_media, A0B1_media, A1B0_media, A1B1_media)
print('El operador de Bell es ', round(A0B0_media+A0B1_media+A1B0_media-A1B1_media, 4))

Vemos que el valor del operador de Bell es notablemente más grande que 2 (de hecho, debería estar cerca del valor $2\sqrt{2}\approx2.8284$). Esto tiene una muy importante consecuencia que ya hemos anticipado antes: **al menos una de las dos premisas del teorema de Bell no se cumplen para sistemas cuánticos**. Esto significa que:

- Los sistemas cuánticos son *no-locales*, es decir, que la medición de un sistema cuántico afecta a todo el resto del universo de manera instantánea, **o**
- Los sistemas cuánticos son *no-realistas*, es decir, que sus propiedades no están definidas antes de realizar una observación, **o**
- Ambas.

La decisión de cuál postulado "sacrificar" depende de la interpretación o la escuela. Por ejemplo, la escuela de Copenhague prefiere sacrificar el postulado de realismo para mantener así cierta coherencia con la teoría de la Relatividad, mientras que la interpretación de de Broglie-Bohm prefiere admitir variables ocultas no-locales para salvar la existencia de una realidad subyacente independiente del proceso de observación. Pero lo que es innegable es que la física cuántica, además de haber revolucionado el mundo de la física y haber permitido el desarrollo tecnológico que comenzó a mediados del siglo XX, nos proporciona una ventana a preguntas fundamentales acerca de las características de la naturaleza y de la realidad que nos rodea.

Para terminar, uno podría argumentar que todos los programas que hemos ejecutado hasta ahora no son más que simulaciones por ordenador, en los cuales lo que se está reflejando son las consecuencias de los postulados de la física cuántica, pero no el comportamiento de sistemas cuánticos reales. En la última parte, haremos uso de los servicios de computación cuántica en la nube puestos a disposición por IBM para mostrar la violación del teorema de Bell utilizando sistemas cuánticos reales.

## Ejecución en ordenadores cuánticos reales
Una buena característica de ```qiskit``` es que acceder a ordenadores cuánticos reales es tan sencillo como utilizar los simuladores clásicos. Hay un pequeño paso extra, que es 'iniciar sesión' en IBM Q Experience. Esto se lleva a cabo a través del módulo ``IBMQ``, y requiere la token asociada a [tu cuenta](https://quantumexperience.ng.bluemix.net/qx/account/advanced).

In [ ]:
QX_TOKEN = ''    # Pega aquí, entre las dos comillas, el API token de tu cuenta
IBMQ.enable_account(QX_TOKEN)

Ahora podemos ver qué ordenadores están disponibles para ejecutar programas.

In [ ]:
provider = IBMQ.get_provider(hub='ibm-q')
print("%20s" % "Name", "|", "%9s" % "N. qubits", "|", "%7s" % "N. Jobs")
print("-------------------------------")
for backend in provider.backends():
    print("%20s" % backend.name(), "|",
          "%9s" % backend.configuration().n_qubits, "|",
          "%7s" % backend.status().pending_jobs)

Y podemos elegir el que nos sea más de gusto.

In [ ]:
chip_real = provider.get_backend('ibmq_ourense')

A continuación, ejecutaremos una versión de los circuitos necesarios para calcular el valor de $\mathcal{B}$ con una pequeña modificación para ganar en velocidad de ejecución: ejecutaremos todas las rondas de un mismo par de preguntas una detrás de otra.

In [ ]:
num_rondas = 1000

ACBC = QuantumCircuit(qr, a, b)
crear_par_entrelazado(ACBC, 0, 1)
ACBC.x(0)
ACBC.measure(0,0)
ACBC.ry(3*π / 4, 1)
ACBC.measure(1,1)

ACBF = QuantumCircuit(qr, a, b)
crear_par_entrelazado(ACBF, 0, 1)
ACBF.x(0)
ACBF.measure(0,0)
ACBF.ry(3*π/4 + π/2, 1)
ACBF.measure(1,1)

AFBC = QuantumCircuit(qr, a, b)
crear_par_entrelazado(AFBC, 0, 1)
AFBC.h(0)
AFBC.x(0)
AFBC.measure(0,0)
AFBC.ry(3*π / 4, 1)
AFBC.measure(1,1)

AFBF = QuantumCircuit(qr, a, b)
crear_par_entrelazado(AFBF, 0, 1)
AFBF.h(0)
AFBF.x(0)
AFBF.measure(0,0)
AFBF.ry(3*π/4 + π/2, 1)
AFBF.measure(1,1)

resultados_ACBC = execute(ACBC, chip_real, shots=num_rondas).result().get_counts()
resultados_ACBF = execute(ACBF, chip_real, shots=num_rondas).result().get_counts()
resultados_AFBC = execute(AFBC, chip_real, shots=num_rondas).result().get_counts()
resultados_AFBF = execute(AFBF, chip_real, shots=num_rondas).result().get_counts()

In [ ]:
ACBC_media = (resultados_ACBC["0 0"] - resultados_ACBC["0 1"] - resultados_ACBC["1 0"] + resultados_ACBC["1 1"]) / num_rondas
ACBF_media = (resultados_ACBF["0 0"] - resultados_ACBF["0 1"] - resultados_ACBF["1 0"] + resultados_ACBF["1 1"]) / num_rondas
AFBC_media = (resultados_AFBC["0 0"] - resultados_AFBC["0 1"] - resultados_AFBC["1 0"] + resultados_AFBC["1 1"]) / num_rondas
AFBF_media = (resultados_AFBF["0 0"] - resultados_AFBF["0 1"] - resultados_AFBF["1 0"] + resultados_AFBF["1 1"]) / num_rondas
print(ACBC_media, ACBF_media, AFBC_media, AFBF_media)
print('El operador de Bell es ', round(ACBC_media+ACBF_media+AFBC_media-AFBF_media, 4))

Y en efecto, volvemos a obtener un valor notablemente mayor que 2. Sin embargo, te habrás dado cuenta de que ahora el valor no es tan grande como antes. Esto ocurre porque los ordenadores cuánticos actuales son aparatos muy sensibles, e incluso una fuente pequeña de ruido afecta de manera significativa las propiedades cuánticas, debilitándolas hasta hacerlas desaparecer. Esto explica por qué, a pesar de que el mundo microscópico se rige por las leyes de la física cuántica, el mundo macroscópico que nosotros experimentamos obedece las leyes de la física clásica desarrolladas por Newton y muchos otros a lo largo de los siglos XVII y XVIII. Sin embargo, la escala exacta y las condiciones en que esto ocurre se mantienen como uno de los grandes problemas sin resolver.

## Conclusiones y más allá
En este taller hemos aprendido las propiedades fundamentales de la física cuántica: la superposición de estados, el colapso de la función de onda, y el entrelazamiento. Además, hemos visto, a través del teorema de Bell, cómo se pueden aprovechar librerías de programación cuántica para desarrollar experimentos, y a ejecutar éstos en plataformas reales a través de servicios en la nube.

Solamente hemos rascado un poco la superficie. Por un lado, porque aún quedan muchos fenómenos por explorar y grandes problemas, incluso con relevancia filosófica como el [problema de la medida](https://en.wikipedia.org/wiki/Measurement_problem), por resolver. Por otro, porque el hecho de que se estén poniendo prototipos de ordenadores cuánticos a disposición del gran público implica que no solamente los físicos, sino cualquiera que lo necesite puede poner a prueba sus hipótesis. A continuación recopilo una lista no exhaustiva de lecturas y recursos de interés:
  - Libros de texto, cursos y lecturas
    - [Quantum Computation and Quantum Information](https://www.cambridge.org/9781107002173), de Michael Nielsen e Isaac Chuang. Poco más que decir que ha sido el libro de texto de referencia desde hace casi ya 20 años.
    - [Learn quantum computation using Qiskit](https://community.qiskit.org/textbook/). Un libro de texto desarrollado por IBM para introducir a la computación cuántica a través de su librería de programación.
    - [Philosophical Issues in Quantum Theory](https://plato.stanford.edu/entries/qt-issues/). Un compendio de los problemas de interés filosófico que la física cuántica pone de manifiesto, como por ejemplo el problema de la medida o la ontología del vector de ondas.
    - [Philosophical aspects of Quantum Information Theory](https://arxiv.org/abs/quant-ph/0611187). Uno de los productos más importantes de la física cuántica a día de hoy es la teoría cuántica de la información. En este texto se formulan distintos temas filosóficos, como la naturaleza de la información y la tesis de Church-Turing, bajo el marco de la teoría cuántica de la información.
  - Plataformas de computación abiertas
    - [IBM Q Experience](https://quantum-computing.ibm.com/). La plataforma de acceso a los ordenadores cuánticos de IBM.
    - [D-Wave Leap](https://cloud.dwavesys.com/leap/). La plataforma de acceso a las máquinas de temple cuántico de D-Wave. Estas máquinas realizan cálculos en otro modelo de computación, diferente al modelo de circuitos, lo cual tiene sus ventajas e inconvenientes.
    - [Azure Quantum](https://azure.microsoft.com/services/quantum). La plataforma de Microsoft para acceder a los ordenadores cuánticos de [IonQ](https://ionq.com/), [Honeywell](https://www.honeywell.com/en-us/company/quantum), y [Quantum Circuits Inc.](https://quantumcircuits.com/)
    - [Braket](https://aws.amazon.com/es/braket/). La plataforma de Amazon para acceder a los ordenadores cuánticos de [Rigetti](https://www.rigetti.com), [D-Wave](https://www.dwavesys.com/) e [IonQ](https://ionq.com/).
  - Librerías de programación para plataformas específicas
    - [Qiskit](www.qiskit.org), para los ordenadores de IBM. Es la que hemos utilizado en este taller, y de hecho es una de las más completas, con una gran cantidad de funcionalidades y de [algoritmos preconstruidos](https://qiskit.org/aqua).
    - [Rigetti Forest](https://www.rigetti.com/forest), para los ordenadores de Rigetti, a los cuales se puede [acceder por invitación](https://www.rigetti.com/qcs).
    - [D-Wave Ocean](https://ocean.dwavesys.com/), para simular y trabajar con las máquinas de D-Wave.
    - [StrawberryFields](https://www.xanadu.ai/software/), de Xanadu AI. Permite simular (y en el futuro, ejecutar en chips reales) ordenadores cuánticos basados en sistemas continuos en lugar de qubits.
    - [Cirq](https://github.com/quantumlib/Cirq), de Google. De momento solo sirve para realizar simulaciones, pero se espera que sera la manera de hacer interfaz con sus procesadores cuando éstos se hagan disponibles al público.
  - Librerías de programación genéricas
    - [Q#](https://www.microsoft.com/en-us/quantum/development-kit). La librería de Microsoft, lo más parecido a C# que se puede ser. Viene acompañada por [un repositorio con tutoriales](https://github.com/Microsoft/QuantumKatas) y programas para problemas comunes.